# Risk Neutral Valuation Analysis

The book claims that the rational pricing for a derivative is NOT given either by:
* a) A free-market where supply and demand come up with a price for the derivative, NOR by
* b) The expected discounted returns of the cashflows from the derivative (i.e. integral over outcomes)

Point b) is surprising to me because that is how bonds are priced, as far as I know. And I expected this to apply also to derivatives, like options. Instead, the rational price is given by Risk-Neutral-Valuation. In this notebook I will inspect what this precisely mean by reproducing and analyzing the example 2.12 of the book.

## Example 2.12 - Pricing a Call Option using Arbitrage
In the example, we price a call option of a stock $S$. The binomial model is used with two possible known outcomes: up and down.

In [12]:
S0 = 100     # initial stock price, t=0
pu = 0.6     # probability of stock going up
pd = 1 - pu  # probability of stock going down
R = 0        # risk-free rate
def S1(up: bool):  # stock price at t=1
    return 120 if up else 80

The expected value of $S_1$ under objective probabilitties is $E^P[S1]$:

In [13]:
E_S1 = pu * S1(up=True) + pd * S1(up=False)  # expected value of S1
E_S1 / (1 + R)  # discounted expected value

104.0